## Toward data-driven topology optimization of concrete structures
**By Pitipat Wongsittikan, SMBT 24'**

*Final project for Creative Machine Learning for Design, Spring 2023*



## 0. Setup
#### Load required packages

In [2]:
# for Topology optimization
using TopOpt, LinearAlgebra, StatsFuns  
# for Data Visualization
using Makie, GLMakie 
# for Data Analysis
using CSV , DataFrames
using Clustering
#These are for the surrogate model
using Flux, Zygote , MLJ
using SurrogatesFlux, Surrogates
using Statistics
using Random
using Distributions

#### Additional settings

In [ ]:
Makie.inline!(true) # so Makie plots are in Jupyter notebook

#### Load and inspect data from the .csv file

In [ ]:
df = CSV.read("Dataset_1.csv", DataFrame)
ndata = size(df)[1]
println("There are $ndata data points in the dataset.")
# println(df[!,"country"])
df_IN = df[df[!,"country"] .== "IN",:];
df_US = df[df[!,"country"] .== "US",:];
df_CA = df[df[!,"country"] .== "CA",:];
df_AU = df[df[!,"country"] .== "AU",:];
df_NZ = df[df[!,"country"] .== "NZ",:];
df_SG = df[df[!,"country"] .== "SG",:];


In [ ]:
#Check headers
foreach(println, names(df))

#### Data Visualization

In [ ]:
f1 = Figure(resolution = (1200, 800)) 
ax1  = Axis(f1[1,1], xlabel = "Strength [MPa]", ylabel = "GWP [kgCO2e/kg]")
ax1.title = "Strength vs GWP" 
# col = df[!,"country"];
scatter!(ax1, df[!,"strength [MPa]"], df[!,"gwp_per_kg [kgCO2e/kg]"], color = :red, markersize = 3)
# Legend(ax1, ["US", "CA", "AU", "NZ", "SG", "IN"])
f1

#### Data preparation
70% for training data 
<br>15% for testing data
<br>15% for validating data

In [ ]:
x_total = collect(df[!,"strength [MPa]"]) ; 
y_total = collect(df[!,"gwp_per_kg [kgCO2e/kg]"]) ;
data = hcat(x_total, y_total); # data is a 2 x n matrix

In [ ]:
data

In [ ]:
train_data, test_data = MLJ.partition(data, 0.7, multi = true, rng = 123)# rng = Random.seed!(1234))


#### Create a neural network surrogate model using Flux

In [ ]:
N1 = Dense(1,1)
N2 = Chain(Dense(1,10,relu), Dense(10,1))
N3 = Chain(Dense(1,10,relu), Dense(10,10,relu), Dense(10,1))
N4 = Chain(Dense(1,10,relu), Dense(10,10,relu), Dense(10,10,relu), Dense(10,1))

models = [N1 , N2 , N3 , N4]

loss1(N1, x,y) = Flux.mse(N1(x),y)
loss2(N2,x,y) = Flux.mse(N2(x),y)
loss3(x,y) = Flux.mse(N3(x),y)
loss4(x,y) = Flux.mse(N4(x),y)

losses = [loss1, loss2, loss3, loss4]

# function my_accuracy( model , )
opt = [Descent(0.01), Descent(0.01), Descent(0.01), Descent(0.01)]
epoch = 500
opt = Adam()



In [ ]:
# my_log = []
for epoch in 1:100
  losses = Float32[]
  for (i, data) in enumerate(train_data)
    println(data)
  end
  break
end
    Flux.train!(loss1, Flux.params(N1), data, opt);
    # model[1].weight
    val = loss1(N1, data)

    # val, grads = Flux.withgradient(N1) do m
      # # Any code inside here is differentiated.
      # # Evaluation of the model and loss must be inside!
      # result = m(input)
      # loss1(result, y)
    end

    # Save the loss from the forward pass. (Done outside of gradient.)
    push!(losses, val)

    # Detect loss of Inf or NaN. Print a warning, and then skip update!
    if !isfinite(val)
      @warn "loss is $val on item $i" epoch
      continue
    end
    # Flux.update!(opt_state, param(N1), grads[1])
  end

  # Compute some accuracy, and save details as a NamedTuple
  # acc = my_accuracy(model, train_set)
  # push!(my_log, (; acc, losses))

  # Stop training when some criterion is reached
  # if  acc > 0.95
  #   println("stopping after $epoch epochs")
  #   break
  # end
end

#### Train models and plot loss.


In [ ]:
loss = Flux.mse